In [2]:
from PIL import Image, ImageFilter
from google.cloud import vision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import base64
import io
import os

#Funciones generales para uso global
import numpy as np
import time
from shapely.geometry import Polygon, Point
import geopandas as gpd
from shapely.ops import cascaded_union
import rasterio
from rasterio.mask import mask
from shapely import geometry
import pandas as pd
from geotiff import GeoTiff


#Funciones empleadas en la detección de texto
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

import rasterio
from rasterio.enums import Resampling

# Model v1: Split the image, detect text areas, and convert to text strings

## Splitting the image

In [63]:
# Takes a Rasterio dataset and splits it into squares of dimensions squareDim * squareDim
def splitImageIntoCells(img, filename, num_imgs=3):
    squareDim_wide = img.shape[1] // num_imgs
    squareDim_height = img.shape[0] // num_imgs
    
    numberOfCellsWide = img.shape[1] // squareDim_wide
    numberOfCellsHigh = img.shape[0] // squareDim_height
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim_height
        for wc in range(numberOfCellsWide):
            x = wc * squareDim_wide
            geom = getTileGeom(img.transform, x, y, squareDim_wide, squareDim_height)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1

# Generate a bounding box from the pixel-wise coordinates using the original datasets transform property
def getTileGeom(transform, x, y, squareDim_wide, squareDim_height):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim_wide, y + squareDim_height) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])

# Crop the dataset using the generated box and write it out as a GeoTIFF
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+"_"+str(count))

# Write the passed in dataset as a GeoTIFF
def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": 'EPSG:4686'})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [6]:
def detect_text(bytes_img):
    """
    Uses the Google Vision API to extract text from
    an image.
    
    Arguments
    ---------
    file_path: str
               path of the image to process.
    
    Outputs
    -------
    response: AnnotateImageResponse object
              json like format with bounding box and other
              relevant information.
    text: str
          text extracted from the image.
    """
    client = vision.ImageAnnotatorClient()    
    image = vision.Image(content=bytes_img)
    response = client.document_text_detection(image=image)
    text = response.full_text_annotation.text
    
    return response, text


def getbytesimg_from_path(filepath):
    """
    Obtains the bytes base64 format of an image from a 
    local file path.
    
    Arguments
    ---------
    filepath: str
              Path of the image file to convert
    
    Output
    ------
    bytes_img: bytes
               base64 format of the image
    """
    with open(filepath, "rb") as image_file:
        bytes_img = image_file.read()
    
    return bytes_img

In [92]:
def split_images(filepath, num_imgs=3):
    """
    Split a large image into a grid of 3x3
    smaller images.
    
    Arguments:
    ---------
    filepath: str
              file path of the large image
    num_imgs: int (optional)
              Number of rows and columns of the grid
              
    Output
    ------
    None
    
    """
    img = rasterio.open(filepath)
    splitImageIntoCells(img, "split_out/output_data")

In [145]:
def geotif_to_jpeg(tif_filename):
    """
    Converts geotif image to jpeg.
    
    Arguments
    
    """
    with rasterio.open(tif_filename) as infile:    
        profile = infile.profile    
        profile['driver']='JPEG'
        jpeg_filename = tif_filename.replace(".tif", ".jpeg")
        with rasterio.open(jpeg_filename, 'w', **profile) as dst:
            dst.write(infile.read())

In [176]:
def get_text_coords(response):
    palabras_google = []
    boundings_google = []
    confidence_google = []
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            palabra_google = ''
            boundings_google.append(block.bounding_box)
            confidence_google.append(block.confidence)
            for parrafos in block.paragraphs:
                for palabras in parrafos.words:
                    for simbolo in palabras.symbols:
                        palabra_google = palabra_google+simbolo.text
                    palabra_google = palabra_google+' '
            palabras_google.append(palabra_google.rstrip())
            
    return palabras_google, boundings_google, confidence_google

In [168]:
def get_coords(boundings, word_index, point_index, coord):
    if coord == "x":
        return boundings[word_index].vertices[point_index].x
    else:
        return boundings[word_index].vertices[point_index].y

In [150]:
split_images("geotiffs/M-1390 F-42290.tif")
geotif_to_jpeg("split_out/output_data_5.tif")
response, text = detect_text(getbytesimg_from_path("split_out/output_data_5.jpeg"))

<ipython-input-63-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-63-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


In [177]:
words, boundings, confidence = get_text_coords(response)

In [179]:
img = rasterio.open("split_out/output_data_5.jpeg")
geometries = []
centroids = []

for i in range(len(words)):
    aux_polygon = Polygon([img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x")),
                           img.xy(get_coords(boundings,i,1,"y"),get_coords(boundings,i,1,"x")),
                           img.xy(get_coords(boundings,i,2,"y"),get_coords(boundings,i,2,"x")),
                           img.xy(get_coords(boundings,i,3,"y"),get_coords(boundings,i,3,"x")),
                           img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x"))])
    geometries.append(aux_polygon)
    centroids.append(aux_polygon.representative_point())

point = gpd.GeoDataFrame()
point["geometry"] = geometries
point["toponimo_ocr"] = words
point["confidence"] = confidence
point["centroide_longitud"] = [x.coords[0][0] for x in centroids]
point["centroide_latitud"] = [x.coords[0][1] for x in centroids]
point.to_file("geometries/output_data_5.geojson", driver="GeoJSON")

In [175]:
response.full_text_annotation.pages[0].blocks[0].confidence

0.8299999833106995

In [180]:
point

,geometry,toponimo_ocr,confidence,centroide_longitud,centroide_latitud
0,"POLYGON ((-75.71607 4.07390, -75.70047 4.07329...",M - 13797-39714,0.83,-75.708319,4.072324
1,"POLYGON ((-75.72643 4.06854, -75.72360 4.07394...",RRAYANES,0.98,-75.724361,4.070891
2,"POLYGON ((-75.72397 4.06358, -75.71832 4.06276...",LAGUNA . EL OASIS,0.92,-75.721352,4.061781
3,"POLYGON ((-75.70817 4.05425, -75.70911 4.04721...",altbę,0.77,-75.710031,4.050911
4,"POLYGON ((-75.72229 4.04573, -75.71853 4.04807...",INOCO,0.47,-75.720083,4.046366


<ipython-input-63-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-63-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


In [107]:
response, text = detect_text(getbytesimg_from_path("test.jpeg"))
text

'GARRAGAN... -\n28/\nG\nGus Y ABAL\nBAL\nOZ OZ -\n'

In [108]:
response.text_annotations[0].bounding_poly

vertices {
  x: 259
  y: 360
}
vertices {
  x: 730
  y: 360
}
vertices {
  x: 730
  y: 830
}
vertices {
  x: 259
  y: 830
}

In [46]:
rectangle = gpd.GeoDataFrame()
rectangle["geometry"] = [Polygon([img.xy(101,383),img.xy(722,383),img.xy(722,593),
         img.xy(101,593),img.xy(101,383)])]
rectangle.to_file("test_geometry.geojson", driver="GeoJSON")

In [73]:
response.full_text_annotation.pages[0].blocks

[property {
  detected_languages {
    language_code: "fil"
    confidence: 1.0
  }
}
bounding_box {
  vertices {
    x: 981
    y: 730
  }
  vertices {
    x: 1646
    y: 622
  }
  vertices {
    x: 1656
    y: 684
  }
  vertices {
    x: 991
    y: 792
  }
}
paragraphs {
  property {
    detected_languages {
      language_code: "fil"
      confidence: 1.0
    }
  }
  bounding_box {
    vertices {
      x: 981
      y: 730
    }
    vertices {
      x: 1646
      y: 622
    }
    vertices {
      x: 1656
      y: 684
    }
    vertices {
      x: 991
      y: 792
    }
  }
  words {
    property {
      detected_languages {
        language_code: "fil"
      }
    }
    bounding_box {
      vertices {
        x: 981
        y: 730
      }
      vertices {
        x: 1149
        y: 703
      }
      vertices {
        x: 1159
        y: 764
      }
      vertices {
        x: 991
        y: 791
      }
    }
    symbols {
      property {
        detected_languages {
          langua

In [38]:
response

text_annotations {
  locale: "es"
  description: "VOZ DE CHAPARRAL\nTRANSMISORES\nBARRID TILUNI\n"
  bounding_poly {
    vertices {
      x: 101
      y: 383
    }
    vertices {
      x: 722
      y: 383
    }
    vertices {
      x: 722
      y: 593
    }
    vertices {
      x: 101
      y: 593
    }
  }
}
text_annotations {
  description: "VOZ"
  bounding_poly {
    vertices {
      x: 101
      y: 443
    }
    vertices {
      x: 169
      y: 433
    }
    vertices {
      x: 175
      y: 475
    }
    vertices {
      x: 107
      y: 485
    }
  }
}
text_annotations {
  description: "DE"
  bounding_poly {
    vertices {
      x: 190
      y: 430
    }
    vertices {
      x: 243
      y: 422
    }
    vertices {
      x: 249
      y: 463
    }
    vertices {
      x: 196
      y: 471
    }
  }
}
text_annotations {
  description: "CHAPARRAL"
  bounding_poly {
    vertices {
      x: 262
      y: 420
    }
    vertices {
      x: 514
      y: 383
    }
    vertices {
      x: 520


In [48]:
rectangle = gpd.GeoDataFrame()
rectangle["geometry"] = [Polygon([img.xy(551,339),img.xy(547,722),img.xy(589,722),
         img.xy(593,339),img.xy(551,339)])]
rectangle.to_file("test_geometry3.geojson", driver="GeoJSON")

In [49]:
rectangle = gpd.GeoDataFrame()
rectangle["geometry"] = [Polygon([img.xy(443,101),img.xy(383,515),img.xy(463,526),
         img.xy(523,113),img.xy(443,101)])]
rectangle.to_file("test_geometry4.geojson", driver="GeoJSON")

In [50]:
response

text_annotations {
  locale: "es"
  description: "VOZ DE CHAPARRAL\nTRANSMISORES\nBARRID TILUNI\n"
  bounding_poly {
    vertices {
      x: 101
      y: 383
    }
    vertices {
      x: 722
      y: 383
    }
    vertices {
      x: 722
      y: 593
    }
    vertices {
      x: 101
      y: 593
    }
  }
}
text_annotations {
  description: "VOZ"
  bounding_poly {
    vertices {
      x: 101
      y: 443
    }
    vertices {
      x: 169
      y: 433
    }
    vertices {
      x: 175
      y: 475
    }
    vertices {
      x: 107
      y: 485
    }
  }
}
text_annotations {
  description: "DE"
  bounding_poly {
    vertices {
      x: 190
      y: 430
    }
    vertices {
      x: 243
      y: 422
    }
    vertices {
      x: 249
      y: 463
    }
    vertices {
      x: 196
      y: 471
    }
  }
}
text_annotations {
  description: "CHAPARRAL"
  bounding_poly {
    vertices {
      x: 262
      y: 420
    }
    vertices {
      x: 514
      y: 383
    }
    vertices {
      x: 520


In [51]:
type(response)

google.cloud.vision_v1.types.image_annotator.AnnotateImageResponse

In [54]:
coord = Polygon([img.xy(551,339),img.xy(547,722),img.xy(589,722),
         img.xy(593,339),img.xy(551,339)]).representative_point()

coord.to_file("test_geometry4.geojson", driver="GeoJSON")

In [58]:
type(coord[0])

tuple

In [57]:
point = gpd.GeoDataFrame()
point["geometry"] = [Polygon([img.xy(551,339),img.xy(547,722),img.xy(589,722),
         img.xy(593,339),img.xy(551,339)]).representative_point()]
point.to_file("test_geometry5.geojson", driver="GeoJSON")

In [60]:
type(coord[0][0])

float

---

In [74]:
filepath = 'geotiffs/C-2070 F-252.tif'
img = rasterio.open(filepath)
splitImageIntoCells(img, 'split_out/salida_data')

<ipython-input-63-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-63-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


In [75]:
tif_filename = "split_out/salida_data_1.tif"
with rasterio.open(tif_filename) as infile:    
    profile=infile.profile    
    profile['driver']='JPEG'
    jpeg_filename="test.jpeg"
    with rasterio.open(jpeg_filename, 'w', **profile) as dst:
        dst.write(infile.read())

In [76]:
response, text = detect_text(getbytesimg_from_path("test.jpeg"))
text

'C-2070 71350\nQULE BRADA\nL\nS\n'

In [91]:
response.full_text_annotation.pages[0].blocks[0]

property {
  detected_languages {
    language_code: "en"
    confidence: 1.0
  }
}
bounding_box {
  vertices {
    x: 256
    y: 356
  }
  vertices {
    x: 603
    y: 374
  }
  vertices {
    x: 599
    y: 444
  }
  vertices {
    x: 252
    y: 426
  }
}
paragraphs {
  property {
    detected_languages {
      language_code: "en"
      confidence: 1.0
    }
  }
  bounding_box {
    vertices {
      x: 256
      y: 356
    }
    vertices {
      x: 603
      y: 374
    }
    vertices {
      x: 599
      y: 444
    }
    vertices {
      x: 252
      y: 426
    }
  }
  words {
    property {
      detected_languages {
        language_code: "en"
      }
    }
    bounding_box {
      vertices {
        x: 256
        y: 357
      }
      vertices {
        x: 272
        y: 358
      }
      vertices {
        x: 268
        y: 427
      }
      vertices {
        x: 252
        y: 426
      }
    }
    symbols {
      property {
        detected_languages {
          language_code: "

In [87]:
text

'C-2070 71350\nQULE BRADA\nL\nS\n'

In [109]:
response.full_text_annotation.pages[0].blocks

[property {
  detected_languages {
    language_code: "en"
    confidence: 1.0
  }
}
bounding_box {
  vertices {
    x: 259
    y: 381
  }
  vertices {
    x: 439
    y: 360
  }
  vertices {
    x: 444
    y: 406
  }
  vertices {
    x: 264
    y: 427
  }
}
paragraphs {
  property {
    detected_languages {
      language_code: "en"
      confidence: 1.0
    }
  }
  bounding_box {
    vertices {
      x: 259
      y: 381
    }
    vertices {
      x: 439
      y: 360
    }
    vertices {
      x: 444
      y: 406
    }
    vertices {
      x: 264
      y: 427
    }
  }
  words {
    property {
      detected_languages {
        language_code: "en"
      }
    }
    bounding_box {
      vertices {
        x: 259
        y: 381
      }
      vertices {
        x: 376
        y: 368
      }
      vertices {
        x: 381
        y: 413
      }
      vertices {
        x: 264
        y: 427
      }
    }
    symbols {
      property {
        detected_languages {
          language_code: 

In [110]:
response, text = detect_text(getbytesimg_from_path("ocr_sample_imgs/sample11.jpg"))
text

'HDA Los\nANDES\nX\nPARAMO LOS ANDES\n'

In [122]:
response.text_annotations[0].description

'HDA Los\nANDES\nX\nPARAMO LOS ANDES\n'

In [123]:
response.full_text_annotation.text

'HDA Los\nANDES\nX\nPARAMO LOS ANDES\n'

In [128]:
response.full_text_annotation.pages[0].blocks[0].bounding_box

vertices {
  x: 57
  y: 31
}
vertices {
  x: 140
  y: 30
}
vertices {
  x: 141
  y: 64
}
vertices {
  x: 58
  y: 65
}

In [139]:
palabras_google

['HDA Los ANDES', 'X PARAMO LOS ANDES']

In [141]:
boundings_google

[vertices {
   x: 57
   y: 31
 }
 vertices {
   x: 140
   y: 30
 }
 vertices {
   x: 141
   y: 64
 }
 vertices {
   x: 58
   y: 65
 },
 vertices {
   x: 20
   y: 278
 }
 vertices {
   x: 264
   y: 277
 }
 vertices {
   x: 264
   y: 321
 }
 vertices {
   x: 20
   y: 322
 }]

In [181]:
type(response)

google.cloud.vision_v1.types.image_annotator.AnnotateImageResponse

In [182]:
os.listdir(path='split_out')

['output_data_5.jpeg',
 '.DS_Store',
 'output_data_0.tif',
 'output_data_1.tif',
 'output_data_3.tif',
 'output_data_2.tif',
 'output_data_6.tif',
 'output_data_7.tif',
 'output_data_5.tif',
 'output_data_4.tif',
 'output_data_8.tif',
 'output_data_5.jpeg.aux.xml']

In [183]:
os.listdir()

['text_detected.geojson',
 'OCR pytesseract.ipynb',
 'Model v1.ipynb',
 '.DS_Store',
 'ocr_sample_imgs',
 'test.jpeg',
 'output_data_6.geojson',
 'response.pickle',
 'test_geometry4.geojson',
 'Model v2.ipynb',
 'response.pkl',
 'split_out',
 'test_geometry5.geojson',
 'filestore.pkl',
 'output_data_5.geojson',
 'test.jpeg.aux.xml',
 'ocr_vision.ipynb',
 'frozen_east_text_detection.pb',
 'geometries',
 '.venv',
 'geotiffs',
 'text_detection_v4.ipynb',
 'text_detection_v3.ipynb',
 '.ipynb_checkpoints',
 'Ejemplos_geotiff_geojson',
 'test_geometry3.geojson',
 'sample_aerophotos',
 'response_example.txt',
 'text_detection_v5.ipynb',
 'test_geometry.geojson',
 'test_geometry2.geojson']

In [184]:
import glob

In [186]:
files = glob.glob("split_out/*")
for f in files:
    os.remove(f)

In [194]:
path = ["corners_img1.json","img1.jpg","text_detected.json_img1.json"]
path

['corners_img1.json', 'img1.jpg', 'text_detected.json_img1.json']

In [195]:
path.sort()

In [196]:
path

['corners_img1.json', 'img1.jpg', 'text_detected.json_img1.json']

In [201]:
for r,d,f in os.walk("geotiffs"):
    print(r)
    print(d)
    print(f)

geotiffs
[]
['.DS_Store', 'M-1390 F-42286.tif', 'M-1390 F-42290.tif', 'C-1974 F-238.tif', 'C-2070 F-252.tif', 'C-2070 F-250.tif', 'C-1974 F-240.tif']


In [202]:
os.listdir()

['Model v3.ipynb',
 'text_detected.geojson',
 'ml_model',
 'OCR pytesseract.ipynb',
 'Model v1.ipynb',
 '.DS_Store',
 'ocr_sample_imgs',
 'response.pickle',
 'Model EAST.ipynb',
 'Model v2.ipynb',
 'response.pkl',
 'split_out',
 'filestore.pkl',
 '__pycache__',
 'test.py',
 'model1.py',
 'results',
 'ocr_vision.ipynb',
 'frozen_east_text_detection.pb',
 'geometries',
 '.venv',
 'geotiffs',
 'text_detection_v4.ipynb',
 'text_detection_v3.ipynb',
 '.ipynb_checkpoints',
 'Ejemplos_geotiff_geojson',
 'sample_aerophotos',
 'response_example.txt',
 'text_detection_v5.ipynb']

In [206]:
glob.glob("geo*")

['geometries', 'geotiffs']

In [211]:
(os.path.abspath("")) + "img_to_process.tif"

'/Users/sergio/Documents/GitHub/IGAC-team18/Modelimg_to_process.tif'

In [213]:
type(eval("""{"path":"/Users/sergio/Documents/GitHub/team18-back-end-app/model"}"""))

dict

In [214]:
file = "mimgg.jpeg"
file.split(".")[1]

'jpeg'

In [216]:
".jpeg" in file

True